In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install SimpleITK
!wget https://github.com/IAmSuyogJadhav/3d-mri-brain-tumor-segmentation-using-autoencoder-regularization/raw/master/model.py

In [0]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
from model_1 import build_model
import glob
from scipy.ndimage import zoom
import re
import nibabel
import os
import time
import math

import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

In [0]:
def read_img(img_path):
  return sitk.GetArrayFromImage(sitk.ReadImage(img_path))

In [0]:
def resize(img, shape, mode='constant', orig_shape=(155, 240, 240)):
  assert len(shape) == 3, "Cannot have more than three dimensions"
  factors = (
      shape[0]/orig_shape[0],
      shape[1]/orig_shape[1],
      shape[2]/orig_shape[2]
  )
  return zoom(img, factors, mode=mode)

def preprocess(img, out_shape=None):
  if out_shape is not None:
    img = resize(img, out_shape, mode='constant')
  
  mean = img.mean()
  std = img.std()
  return (img - mean)/std

def preprocess_label(img, out_shape=None, mode='nearest'):
  ncr = img == 1
  ed = img == 2
  et = img == 4

  if out_shape is not None:
    ncr = resize(ncr, out_shape, mode=mode)
    ed = resize(ed, out_shape, mode=mode)
    et = resize(et, out_shape, mode=mode)
  
  return np.array([ncr, ed, et], dtype=np.uint8)

In [0]:
t1 = glob.glob(r'/content/drive/My Drive/BRATS2019/BRATS19/MICCAI_BraTS_2019_Data_Training/HGG/*/*t1.nii.gz')
t2 = glob.glob(r'/content/drive/My Drive/BRATS2019/BRATS19/MICCAI_BraTS_2019_Data_Training/HGG/*/*t2.nii.gz')
flair = glob.glob(r'/content/drive/My Drive/BRATS2019/BRATS19/MICCAI_BraTS_2019_Data_Training/HGG/*/*flair.nii.gz')
t1ce = glob.glob(r'/content/drive/My Drive/BRATS2019/BRATS19/MICCAI_BraTS_2019_Data_Training/HGG/*/*t1ce.nii.gz')
seg = glob.glob(r'/content/drive/My Drive/BRATS2019/BRATS19/MICCAI_BraTS_2019_Data_Training/HGG/*/*seg.nii.gz')

In [0]:
pat = re.compile('.*_(\w*)\.nii\.gz')

data_paths = [{
    pat.findall(item)[0]:item
    for item in items
}
for items in list(zip(t1, t2, t1ce, flair, seg))]

In [0]:
input_shape = (4,80,96,64)
output_channels = 3
data = np.empty((len(data_paths[:4]),)+input_shape, dtype=np.float32)
labels = np.empty((len(data_paths[:4]),output_channels)+input_shape[1:], dtype=np.uint8)

In [0]:
timee = time.time()

for i, imgs in enumerate(data_paths[:4]):
  try:
    labels[i] = preprocess_label(read_img(imgs['seg']), input_shape[1:])[None, ...]
    data[i] = np.array([preprocess(read_img(imgs[m]), input_shape[1:]) for m in ['t1', 't2', 't1ce', 'flair']], dtype=np.float32)
    print(i)

  except Exception as e:
    print(f'Exception: {str(e)}')
    continue

print(time.time() - timee)

In [0]:
model = build_model(input_shape=input_shape, output_channels=3)

In [0]:
timee = time.time()
model.fit(data, labels, batch_size=1, validation_split=0.2, epochs=15)
print(time.time()-timee)

In [0]:
model.save('/gdrive/My Drive/BRATS2019/seg_model.h5')

In [0]:
df = pd.read_csv('training_data_classification_labels.csv')
column = "CPM_RadPath_2019_ID"
classy = "class"

In [0]:
data_path = "/gdrive/My Drive/UPenn_Data/miccai2019-data/CPM-RadPath_2019_Training_Data/Radiology"
flair = "_flair.nii.gz"
t1 = "_t1.nii.gz"
t1_ce = "_t1ce.nii.gz"
t2 = "_t2.nii.gz"

In [0]:
input_shape_test = (4,80,96,64)
output_channels_test = 3

In [0]:
t1_path = []
t1ce_path = []
flair_path = []
t2_path = []
def data_paths():
    for i in range(df[column].shape[0]):
        data_path2 = df[column][i]
        t2_path.append(os.path.join(data_path,data_path2,data_path2+t2))
        flair_path.append(os.path.join(data_path,data_path2,data_path2+flair))
        t1_path.append(os.path.join(data_path,data_path2,data_path2+t1))
        t1ce_path.append(os.path.join(data_path,data_path2,data_path2+t1_ce))
    return

data_paths()

In [0]:
pat_test = re.compile('.*_(\w*)\.nii\.gz')

data_paths_test = [{
    pat_test.findall(item)[0]:item
    for item in items
}
for items in list(zip(t1_path, t2_path, t1ce_path, flair_path))]
print(data_paths_test)

In [0]:
timee = time.time()
data_test = np.empty((221,)+input_shape_test, dtype=np.float32)
x_in = 20
y_in = 50
for i,imgs in enumerate(data_paths_test[:]):
    try:
        data_test[i] = np.array([preprocess(read_img(imgs[m]), input_shape_test[1:]) for m in ['t1', 't2', 't1ce', 'flair']], dtype=np.float32)
        print(i)
    
    except Exception as e:
        print(f'Exception: {str(e)}')
        continue

print(time.time()-timee)

In [0]:
lis = []
lis = model.predict(data_test)

out_path = "/gdrive/My Drive/UPenn_Data/miccai2019-data/CPM-RadPath_2019_Training_Data/Radiology"

timee = time.time()
for i in range(0,221):
  try:  
    pathss = df[column][i]
    img1 = nib.Nifti1Image(lis[i][0], affine=np.eye(4))
    img2 = nib.Nifti1Image(lis[i][1], affine=np.eye(4))
    img3 = nib.Nifti1Image(lis[i][2], affine=np.eye(4))

    nib.save(img1, os.path.join(out_path,pathss,pathss+"_img1.nii.gz"))
    nib.save(img2, os.path.join(out_path,pathss,pathss+"_img2.nii.gz"))
    nib.save(img3, os.path.join(out_path,pathss,pathss+"_img3.nii.gz"))
    print(i)
  
  except Exception as e:
    print(f'Exception: str{e}')
    continue

print(time.time()-timee)